In [5]:
import sys, os

# Chemin vers le dossier src de ton projet
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../music-recommender"))
src_path = os.path.join(project_root, "src")

# Ajoute le chemin au PYTHONPATH si pas déjà présent
if src_path not in sys.path:
    sys.path.append(src_path)
    
print("✅ Path ajouté :", src_path)


from ingestion.scrapping import (
    generate_album_links,
    scrape_url,
)

from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
print("✅ Import réussi !")


✅ Path ajouté : c:\Users\artmu\Desktop\Arthur\Etude\Etude_sup\IUT\Cours\3A\S5\IA\music-recommender\src
✅ Import réussi !


In [6]:
urls = generate_album_links(00000, 25000)

output_path = "../data/processed/sample_albums.csv"

if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    scraped_urls = set(df_existing["source_url"])
    print(f" {len(scraped_urls)} albums déjà présents — ils seront ignorés.")
else:
    scraped_urls = set()
    print(" Aucun dataset existant, scraping complet.")


# === Version multithread conservant la logique de mise à jour ===
urls_to_scrape = [u for u in urls if u not in scraped_urls]
print(f" === Lancement du scraping multithread ({len(urls_to_scrape)} nouvelles URLs)... === ")

rows = []
max_threads = min(32, (os.cpu_count() or 1) * 2) # adapter selon mon CPU / réseau

with ThreadPoolExecutor(max_workers=max_threads) as executor:
    futures = {executor.submit(scrape_url, url): url for url in urls_to_scrape}
    for i, future in enumerate(as_completed(futures), 1):
        result = future.result()
        if result:
            rows.append(result)
            print(f"[{i}] {result['artist_name']} - {result['album_name']}")
        else:
            print(f"[{i}] Échec ou page vide")

        # Sauvegarde intermédiaire tous les 100 albums valides
        if i % 100 == 0 and rows:
            temp_df = pd.DataFrame(rows)
            if os.path.exists(output_path):
                df_existing = pd.read_csv(output_path)
                temp_df = pd.concat([df_existing, temp_df], ignore_index=True)
                temp_df.drop_duplicates(subset=["source_url"], inplace=True)
            temp_df.to_csv(output_path, index=False, encoding="utf-8")
            print(f" === Sauvegarde intermédiaire ({len(temp_df)} albums) → {output_path} ===")
            rows = []  # on vide la mémoire

df = pd.DataFrame(rows)
print("\n=== Aperçu des données ===")
display(df.head())

if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    df = pd.concat([df_existing, df], ignore_index=True)

# Nettoyage avant export
df.drop_duplicates(subset=["source_url"], inplace=True)
df.reset_index(drop=True, inplace=True)

df.to_csv(output_path, index=False, encoding="utf-8")
print(f"\n Données exportées vers {output_path} ({len(df)} albums uniques)")

new_albums_count = len(df) - len(df_existing) if os.path.exists(output_path) else len(df)
print(f"\n {new_albums_count} nouveaux albums ajoutés. Total : {len(df)} albums uniques.")



 25001 liens générés (https://www.gutsofdarkness.com/god/objet.php?objet=0 → https://www.gutsofdarkness.com/god/objet.php?objet=25000)
 9275 albums déjà présents — ils seront ignorés.
 === Lancement du scraping multithread (15726 nouvelles URLs)... === 
[1] Échec ou page vide
[2] Échec ou page vide
[3] Échec ou page vide
[4] Échec ou page vide
[5] Échec ou page vide
[6] Échec ou page vide
[7] Échec ou page vide
[8] Échec ou page vide
[9] Échec ou page vide
[10] Échec ou page vide
[11] Échec ou page vide
[12] Échec ou page vide
[13] Échec ou page vide
[14] Échec ou page vide
[15] Échec ou page vide
[16] Échec ou page vide
[17] Échec ou page vide
[18] Échec ou page vide
[19] Échec ou page vide
[20] Échec ou page vide
[21] Échec ou page vide
[22] Échec ou page vide
[23] Échec ou page vide
[24] Échec ou page vide
[25] Échec ou page vide
[26] Échec ou page vide
[27] Échec ou page vide
[28] Échec ou page vide
[29] Échec ou page vide
[30] Échec ou page vide
[31] Échec ou page vide
[32] Échec 

""



 Données exportées vers ../data/processed/sample_albums.csv (23994 albums uniques)

 0 nouveaux albums ajoutés. Total : 23994 albums uniques.
